# Stacked histogram reading
## and extracting data / MC to get KL divergences

***
# Used packages
For plotting, calculating KL div
***

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
#plt.style.use([hep.style.ROOT, hep.style.fira, hep.style.firamath])
#plt.style.use([hep.style.ROOT, hep.style.firamath])

In [ ]:
plt.style.use([hep.style.ROOT])

In [ ]:
from scipy.stats import entropy

In [ ]:
def kldiv(A, B):
        return np.sum([v for v in A * np.log(A/B) if not np.isnan(v)])
    
def jsdiv(P, Q, normalize_first=False):
    """Compute the Jensen-Shannon divergence between two probability distributions.

    Input
    -----
    P, Q : array-like
        Probability distributions of equal length that sum to 1
    """
    
    P = np.array(P)
    Q = np.array(Q)

    if normalize_first:
        P = P/P.sum()
        Q = Q/Q.sum()
    
    M = 0.5 * (P + Q)

    return 0.5 * (kldiv(P, M) +kldiv(Q, M))

In [ ]:
def chi2(P, Q, normalize_first=False):
    P = np.array(P)
    Q = np.array(Q)
    if normalize_first:
        P = P/P.sum()
        Q = Q/Q.sum()
    
    return np.sum([v for v in ((P-Q)**2 / (Q)) if not np.isnan(v)])

***
# Relevant final implementation starts here (update paths when necessary)
***

### Specify paths for different epochs
StackPlotter for three epochs or one epoch only results in slightly different directory names, but similar file names  
This is the place to add paths up to the point where they start to be similar again

In [ ]:
ana_outdir_adv = ['/nfs/dust/cms/user/anstein/ctag_condor/systPlots_220809_2017_best_adversarial_eps0p01_/Plots_220809_best_adversarial_eps0p01_minimal_Custom']
# with 30bins only for easier comparison even for low stats
#ana_outdir_adv = ['/nfs/dust/cms/user/anstein/ctag_condor/systPlots_220915_2017_best_adversarial_eps0p01_/Plots_220915_best_adversarial_eps0p01_minimal_Custom_30bins']

ana_outdir_adv_dict = {'best':path for path in ana_outdir_adv}

In [ ]:
ana_outdir_basic = ['/nfs/dust/cms/user/anstein/ctag_condor/systPlots_220811_2017_best_nominal_/Plots_220811_best_nominal_minimal_Custom']
# with 30bins only for easier comparison even for low stats
#ana_outdir_basic = ['/nfs/dust/cms/user/anstein/ctag_condor/systPlots_220915_2017_best_nominal_/Plots_220915_best_nominal_minimal_Custom_30bins']
ana_outdir_basic_dict = {'best':path for path in ana_outdir_basic}

Need to match the epochs with letters used

In [ ]:
'''
epoch_letter_dict = {1: '_A_',
                    50: '_B_',
                    150:'_C_',
                     5: '_A_',
                    10: '_B_',
                    100:'_C_',
                    200:''}
'''
epoch_letter_dict = {1: '_A_',
                    50: '_B_',
                    150:'_C_',
                     5: '_A_',
                    10: '_B_',
                    100:'_C_',
                    'best':''}

### Calculate KL divergence for adversarial training

In [ ]:
epochs = ['best']  # add the ones for which StackPlotter results exist
wms = ['DY_m', 'TT_semim', 'Wc_m']
# DY sel
kl_divs_all_adv_DY          = []
js_divs_all_adv_DY          = []
chi2_all_adv_DY          = []
# Wc sel
kl_divs_all_adv_Wc          = []
js_divs_all_adv_Wc          = []
chi2_all_adv_Wc          = []
# TT sel
kl_divs_all_adv_TT          = []
js_divs_all_adv_TT          = []
chi2_all_adv_TT          = []

kl_holder_adv = {'DY_m' : kl_divs_all_adv_DY, 'Wc_m' : kl_divs_all_adv_Wc, 'TT_semim' : kl_divs_all_adv_TT}
js_holder_adv = {'DY_m' : js_divs_all_adv_DY, 'Wc_m' : js_divs_all_adv_Wc, 'TT_semim' : js_divs_all_adv_TT}
chi2_holder_adv = {'DY_m' : chi2_all_adv_DY, 'Wc_m' : chi2_all_adv_Wc, 'TT_semim' : chi2_all_adv_TT}
#kl_divs_all_adv_DY_datadata = []
#kl_divs_all_adv_DY_simsim   = []
#data_disc_histo_dict = {'Prob_b':[],'Prob_bb':[],'Prob_c':[],'Prob_l':[],'BvsL':[], 'BvsC':[], 'CvsB':[], 'CvsL':[]}
#sim_disc_histo_dict = {'Prob_b':[],'Prob_bb':[],'Prob_c':[],'Prob_l':[],'BvsL':[], 'BvsC':[], 'CvsB':[], 'CvsL':[]}
#data_disc_histo_dict = {'BvsL':[], 'BvsC':[], 'CvsB':[], 'CvsL':[]}
#sim_disc_histo_dict = {'BvsL':[], 'BvsC':[], 'CvsB':[], 'CvsL':[]}
for j,wm in enumerate(wms):
    if wm == 'DY_m':
        indexjet = '0'
    else:
        indexjet = 'muJet_idx'
    for i,e in enumerate(epochs):
        kl_divs_adv          = []
        js_divs_adv          = []
        chi2_adv          = []
        #kl_divs_adv_datadata = []
        #kl_divs_adv_simsim   = []
        for disc in ['BvsL', 'CvsB', 'CvsL']:  # select those that exist for paths listed below
            if True:
                # data to MC
                #print(ana_outdir_adv_dict[e])
                histo = uproot.open(ana_outdir_adv_dict[e]+f'/output_2017_PFNano_central/{wm}_jet_Custom{epoch_letter_dict[e]}{disc}_{indexjet}_.root')

                num, denom = histo['Data'].values(), histo['MCSum'].values()
                num_num, denom_num = num, denom
                #print(num, denom)
                denom[(denom == 0) & (num != 0)] = 0.00001
                where_num_or_denom_negative = (denom < 0) | (num < 0)
                num[where_num_or_denom_negative] = 0
                denom[where_num_or_denom_negative] = 0
                #print(num, denom)
                this_kl_a = entropy([v for v in num] , qk=[v for v in denom])
                kl_divs_adv.append(this_kl_a)

                num, denom = histo['Data'].values(), histo['MCSum'].values()
                this_js = jsdiv(num, denom, normalize_first=True)
                this_chi2 = chi2(num, denom, normalize_first=True)

                js_divs_adv.append(this_js)
                chi2_adv.append(this_chi2)

                '''
                # data / data and MC / MC
                #print(data_disc_histo_dict[disc])
                #print(data_disc_histo_dict)
                #print(disc)
                if i > 0:
                    # data / data (this epoch: num_num / previous epoch: look up in dictionary)                
                    data_disc_histo_dict[disc][(data_disc_histo_dict[disc] == 0) & (num_num != 0)] = 0.00001
                    where_num_or_denom_negative = (data_disc_histo_dict[disc] < 0) | (num_num < 0)
                    num_num[where_num_or_denom_negative] = 0
                    data_disc_histo_dict[disc][where_num_or_denom_negative] = 0

                    # mc / mc (this epoch: denom_num / previous epoch: look up in dictionary)                
                    sim_disc_histo_dict[disc][(sim_disc_histo_dict[disc] == 0) & (denom_num != 0)] = 0.00001
                    where_num_or_denom_negative = (sim_disc_histo_dict[disc] < 0) | (denom_num < 0)
                    denom_num[where_num_or_denom_negative] = 0
                    sim_disc_histo_dict[disc][where_num_or_denom_negative] = 0


                    kl_divs_adv_datadata.append(entropy([v for v in num_num] , qk=data_disc_histo_dict[disc]))
                    kl_divs_adv_simsim.append(entropy([v for v in denom_num] , qk=sim_disc_histo_dict[disc]))
                
                # store this epoch's histogram for the respective variable in the dictionary (makes comparing with previous epoch possible)
                data_disc_histo_dict[disc] = num
                sim_disc_histo_dict[disc] = denom
                '''
            if False:
                kl_divs_adv.append(np.NaN)

        #print(kl_divs_adv)
        kl_holder_adv[wm].append(kl_divs_adv)
        js_holder_adv[wm].append(js_divs_adv)
        chi2_holder_adv[wm].append(chi2_adv)
        '''
        if i > 0:
            kl_divs_all_adv_DY_datadata.append(kl_divs_adv_datadata)
            kl_divs_all_adv_DY_simsim.append(kl_divs_adv_simsim)
        '''
    #kl_divs_all_adv_DY = np.array(kl_divs_all_adv_DY)
    #kl_divs_all_adv_DY_datadata = np.array(kl_divs_all_adv_DY_datadata)
    #kl_divs_all_adv_DY_simsim = np.array(kl_divs_all_adv_DY_simsim)

    print(kl_divs_all_adv_DY)
    print(js_divs_all_adv_DY)
    print(chi2_all_adv_DY)
    print(kl_divs_all_adv_Wc)
    print(js_divs_all_adv_Wc)
    print(chi2_all_adv_Wc)
    print(kl_divs_all_adv_TT)
    print(js_divs_all_adv_TT)
    print(chi2_all_adv_TT)
    #print(kl_divs_all_adv_DY_datadata)
    #print(kl_divs_all_adv_DY_simsim)
    print()

In [ ]:
epochs = ['best']  # add the ones for which StackPlotter results exist
wms = ['DY_m', 'TT_semim', 'Wc_m']
# DY sel
kl_divs_all_basic_DY          = []
js_divs_all_basic_DY          = []
chi2_all_basic_DY          = []
# Wc sel
kl_divs_all_basic_Wc          = []
js_divs_all_basic_Wc          = []
chi2_all_basic_Wc          = []
# TT sel
kl_divs_all_basic_TT          = []
js_divs_all_basic_TT          = []
chi2_all_basic_TT          = []

kl_holder_basic = {'DY_m' : kl_divs_all_basic_DY, 'Wc_m' : kl_divs_all_basic_Wc, 'TT_semim' : kl_divs_all_basic_TT}
js_holder_basic = {'DY_m' : js_divs_all_basic_DY, 'Wc_m' : js_divs_all_basic_Wc, 'TT_semim' : js_divs_all_basic_TT}
chi2_holder_basic = {'DY_m' : chi2_all_basic_DY, 'Wc_m' : chi2_all_basic_Wc, 'TT_semim' : chi2_all_basic_TT}
#kl_divs_all_basic_DY_datadata = []
#kl_divs_all_basic_DY_simsim   = []
#data_disc_histo_dict = {'Prob_b':[],'Prob_bb':[],'Prob_c':[],'Prob_l':[],'BvsL':[], 'BvsC':[], 'CvsB':[], 'CvsL':[]}
#sim_disc_histo_dict = {'Prob_b':[],'Prob_bb':[],'Prob_c':[],'Prob_l':[],'BvsL':[], 'BvsC':[], 'CvsB':[], 'CvsL':[]}
#data_disc_histo_dict = {'BvsL':[], 'BvsC':[], 'CvsB':[], 'CvsL':[]}
#sim_disc_histo_dict = {'BvsL':[], 'BvsC':[], 'CvsB':[], 'CvsL':[]}
for j,wm in enumerate(wms):
    if wm == 'DY_m':
        indexjet = '0'
    else:
        indexjet = 'muJet_idx'
    for i,e in enumerate(epochs):
        kl_divs_basic          = []
        js_divs_basic          = []
        chi2_basic          = []
        #kl_divs_basic_datadata = []
        #kl_divs_basic_simsim   = []
        for disc in ['BvsL', 'CvsB', 'CvsL']:  # select those that exist for paths listed below
            if True:
                # data to MC
                #print(ana_outdir_basic_dict[e])
                histo = uproot.open(ana_outdir_basic_dict[e]+f'/output_2017_PFNano_central/{wm}_jet_Custom{epoch_letter_dict[e]}{disc}_{indexjet}_.root')

                num, denom = histo['Data'].values(), histo['MCSum'].values()
                num_num, denom_num = num, denom
                #print(num, denom)
                denom[(denom == 0) & (num != 0)] = 0.00001
                where_num_or_denom_negative = (denom < 0) | (num < 0)
                num[where_num_or_denom_negative] = 0
                denom[where_num_or_denom_negative] = 0
                #print(num, denom)
                this_kl_a = entropy([v for v in num] , qk=[v for v in denom])
                kl_divs_basic.append(this_kl_a)

                num, denom = histo['Data'].values(), histo['MCSum'].values()
                this_js = jsdiv(num, denom, normalize_first=True)
                this_chi2 = chi2(num, denom, normalize_first=True)

                js_divs_basic.append(this_js)
                chi2_basic.append(this_chi2)

                '''
                # data / data and MC / MC
                #print(data_disc_histo_dict[disc])
                #print(data_disc_histo_dict)
                #print(disc)
                if i > 0:
                    # data / data (this epoch: num_num / previous epoch: look up in dictionary)                
                    data_disc_histo_dict[disc][(data_disc_histo_dict[disc] == 0) & (num_num != 0)] = 0.00001
                    where_num_or_denom_negative = (data_disc_histo_dict[disc] < 0) | (num_num < 0)
                    num_num[where_num_or_denom_negative] = 0
                    data_disc_histo_dict[disc][where_num_or_denom_negative] = 0

                    # mc / mc (this epoch: denom_num / previous epoch: look up in dictionary)                
                    sim_disc_histo_dict[disc][(sim_disc_histo_dict[disc] == 0) & (denom_num != 0)] = 0.00001
                    where_num_or_denom_negative = (sim_disc_histo_dict[disc] < 0) | (denom_num < 0)
                    denom_num[where_num_or_denom_negative] = 0
                    sim_disc_histo_dict[disc][where_num_or_denom_negative] = 0


                    kl_divs_basic_datadata.append(entropy([v for v in num_num] , qk=data_disc_histo_dict[disc]))
                    kl_divs_basic_simsim.append(entropy([v for v in denom_num] , qk=sim_disc_histo_dict[disc]))
                
                # store this epoch's histogram for the respective variable in the dictionary (makes comparing with previous epoch possible)
                data_disc_histo_dict[disc] = num
                sim_disc_histo_dict[disc] = denom
                '''
            if False:
                kl_divs_basic.append(np.NaN)

        #print(kl_divs_basic)
        kl_holder_basic[wm].append(kl_divs_basic)
        js_holder_basic[wm].append(js_divs_basic)
        chi2_holder_basic[wm].append(chi2_basic)
        '''
        if i > 0:
            kl_divs_all_basic_DY_datadata.append(kl_divs_basic_datadata)
            kl_divs_all_basic_DY_simsim.append(kl_divs_basic_simsim)
        '''
    #kl_divs_all_basic_DY = np.array(kl_divs_all_basic_DY)
    #kl_divs_all_basic_DY_datadata = np.array(kl_divs_all_basic_DY_datadata)
    #kl_divs_all_basic_DY_simsim = np.array(kl_divs_all_basic_DY_simsim)

    print(kl_divs_all_basic_DY)
    print(js_divs_all_basic_DY)
    print(chi2_all_basic_DY)
    print(kl_divs_all_basic_Wc)
    print(js_divs_all_basic_Wc)
    print(chi2_all_basic_Wc)
    print(kl_divs_all_basic_TT)
    print(js_divs_all_basic_TT)
    print(chi2_all_basic_TT)
    #print(kl_divs_all_basic_DY_datadata)
    #print(kl_divs_all_basic_DY_simsim)
    print()

In [ ]:
#xses = ['BvsL', 'BvsC', 'CvsB', 'CvsL']

In [ ]:
xses = ['BvsL', 'CvsB', 'CvsL']

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,8))
fig.subplots_adjust(wspace = 0.3)
#fig.tight_layout()
ax1.scatter(xses, js_divs_all_basic_DY, label='light jets nominal'   , s=100 )
ax1.scatter(xses, js_divs_all_adv_DY, label='light jets adversarial' , s=100 )
ax2.scatter(xses, js_divs_all_basic_Wc, label='c jets nominal'       , s=100 )
ax2.scatter(xses, js_divs_all_adv_Wc, label='c jets adversarial'     , s=100 )
ax3.scatter(xses, js_divs_all_basic_TT, label='b jets nominal'       , s=100 )
ax3.scatter(xses, js_divs_all_adv_TT, label='b jets adversarial'     , s=100 )
ax1.legend(frameon=True)
ax2.legend(frameon=True)
ax3.legend(frameon=True)
ax1.set_ylabel('JS divergence (a.u.)')
ax2.set_ylabel('JS divergence (a.u.)')
ax3.set_ylabel('JS divergence (a.u.)')
hep.cms.label('Preliminary', data=True, lumi=41.54, year=2017)###fig.suptitle('JS divergence. Best epoch.', y=0.95)
#fig.savefig('js_best_epoch_custom_DeepJet_forRTG.pdf')

In [ ]:
plt.rcParams['axes.axisbelow'] = True

In [ ]:
#fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,8))
fig, ax1 = plt.subplots(1, figsize=(9,8))
#fig.subplots_adjust(wspace = 0.3)
fig.tight_layout()
ax1.grid(which='minor', alpha=0.85)
ax1.grid(which='major', alpha=0.95, color='black')
ax1.scatter(xses, js_divs_all_basic_DY, label='Nominal'   , s=200 )
ax1.scatter(xses, js_divs_all_adv_DY, label='Adversarial' , s=200 )
#ax2.scatter(xses, js_divs_all_basic_Wc, label='c jets nominal'       , s=100 )
#ax2.scatter(xses, js_divs_all_adv_Wc, label='c jets adversarial'     , s=100 )
#ax3.scatter(xses, js_divs_all_basic_TT, label='b jets nominal'       , s=100 )
#ax3.scatter(xses, js_divs_all_adv_TT, label='b jets adversarial'     , s=100 )
#ax1.legend(frameon=True, title='Light jets')
ax1.legend(frameon=True, framealpha=1, title='light jets, DeepJet')
leg = ax1.get_legend()
leg._legend_box.align = "left"
leg.get_frame().set_linewidth(0.0)
#ax2.legend(frameon=True)
#ax3.legend(frameon=True)
ax1.set_ylabel('JS divergence (a.u.)')
#ax2.set_ylabel('JS divergence (a.u.)')
#ax3.set_ylabel('JS divergence (a.u.)')
hep.cms.label('Preliminary', data=True, lumi=41.5, year=2017)###fig.suptitle('JS divergence. Best epoch.', y=0.95)
fig.savefig('js_best_epoch_custom_DeepJet_LIGHT_v2.pdf', bbox_inches='tight')
#fig.savefig('js_best_epoch_custom_DeepJet_LIGHT_v2_30bins.pdf', bbox_inches='tight')

In [ ]:
#fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,8))
fig, ax1 = plt.subplots(1, figsize=(9,8))
#fig.subplots_adjust(wspace = 0.3)
fig.tight_layout()
ax1.grid(which='minor', alpha=0.85)
ax1.grid(which='major', alpha=0.95, color='black')
ax1.scatter(xses, chi2_all_basic_DY, label='Nominal'   , s=200 )
ax1.scatter(xses, chi2_all_adv_DY, label='Adversarial' , s=200 )
#ax2.scatter(xses, js_divs_all_basic_Wc, label='c jets nominal'       , s=100 )
#ax2.scatter(xses, js_divs_all_adv_Wc, label='c jets adversarial'     , s=100 )
#ax3.scatter(xses, js_divs_all_basic_TT, label='b jets nominal'       , s=100 )
#ax3.scatter(xses, js_divs_all_adv_TT, label='b jets adversarial'     , s=100 )
#ax1.legend(frameon=True, title='Light jets')
ax1.legend(frameon=True, framealpha=1, title='light jets, DeepJet')
leg = ax1.get_legend()
leg._legend_box.align = "left"
leg.get_frame().set_linewidth(0.0)
#ax2.legend(frameon=True)
#ax3.legend(frameon=True)
ax1.set_ylabel(r'$\chi^2$ (a.u.)')
#ax2.set_ylabel('JS divergence (a.u.)')
#ax3.set_ylabel('JS divergence (a.u.)')
hep.cms.label('Preliminary', data=True, lumi=41.5, year=2017)###fig.suptitle('JS divergence. Best epoch.', y=0.95)
fig.savefig('chi2_best_epoch_custom_DeepJet_LIGHT_v2.pdf', bbox_inches='tight')
#fig.savefig('chi2_best_epoch_custom_DeepJet_LIGHT_v2_30bins.pdf', bbox_inches='tight')

In [ ]:
#fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,8))
fig, ax2 = plt.subplots(1, figsize=(9,8))
#fig.subplots_adjust(wspace = 0.3)
fig.tight_layout()
ax2.grid(which='minor', alpha=0.85)
ax2.grid(which='major', alpha=0.95, color='black')
#ax1.scatter(xses, js_divs_all_basic_DY, label='light jets nominal'   , s=100 )
#ax1.scatter(xses, js_divs_all_adv_DY, label='light jets adversarial' , s=100 )
ax2.scatter(xses, js_divs_all_basic_Wc, label='Nominal'       , s=200 )
ax2.scatter(xses, js_divs_all_adv_Wc, label='Adversarial'     , s=200 )
#ax3.scatter(xses, js_divs_all_basic_TT, label='b jets nominal'       , s=100 )
#ax3.scatter(xses, js_divs_all_adv_TT, label='b jets adversarial'     , s=100 )
#ax1.legend(frameon=True)
ax2.legend(frameon=True, framealpha=1, title='c jets, DeepJet', loc="upper left")
leg = ax2.get_legend()
leg._legend_box.align = "left"
leg.get_frame().set_linewidth(0.0)
#ax3.legend(frameon=True)
#ax1.set_ylabel('JS divergence (a.u.)')
ax2.set_ylabel('JS divergence (a.u.)')
#ax3.set_ylabel('JS divergence (a.u.)')
hep.cms.label('Preliminary', data=True, lumi=41.5, year=2017)###fig.suptitle('JS divergence. Best epoch.', y=0.95)
fig.savefig('js_best_epoch_custom_DeepJet_CHARM_v2.pdf', bbox_inches='tight')
#fig.savefig('js_best_epoch_custom_DeepJet_CHARM_v2_30bins.pdf', bbox_inches='tight')

In [ ]:
#fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,8))
fig, ax2 = plt.subplots(1, figsize=(9,8))
#fig.subplots_adjust(wspace = 0.3)
fig.tight_layout()
ax2.grid(which='minor', alpha=0.85)
ax2.grid(which='major', alpha=0.95, color='black')
#ax1.scatter(xses, js_divs_all_basic_DY, label='light jets nominal'   , s=100 )
#ax1.scatter(xses, js_divs_all_adv_DY, label='light jets adversarial' , s=100 )
ax2.scatter(xses, chi2_all_basic_Wc, label='Nominal'       , s=200 )
ax2.scatter(xses, chi2_all_adv_Wc, label='Adversarial'     , s=200 )
#ax3.scatter(xses, js_divs_all_basic_TT, label='b jets nominal'       , s=100 )
#ax3.scatter(xses, js_divs_all_adv_TT, label='b jets adversarial'     , s=100 )
#ax1.legend(frameon=True)
ax2.legend(frameon=True, framealpha=1, title='c jets, DeepJet', loc="upper right")
leg = ax2.get_legend()
leg._legend_box.align = "right"
leg.get_frame().set_linewidth(0.0)
#ax3.legend(frameon=True)
#ax1.set_ylabel('JS divergence (a.u.)')
ax2.set_ylabel(r'$\chi^2$ (a.u.)')
#ax3.set_ylabel('JS divergence (a.u.)')
hep.cms.label('Preliminary', data=True, lumi=41.5, year=2017)###fig.suptitle('JS divergence. Best epoch.', y=0.95)
fig.savefig('chi2_best_epoch_custom_DeepJet_CHARM_v2.pdf', bbox_inches='tight')
#fig.savefig('chi2_best_epoch_custom_DeepJet_CHARM_v2_30bins.pdf', bbox_inches='tight')

In [ ]:
#fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,8))
fig, ax3 = plt.subplots(1, figsize=(9,8))
#fig.subplots_adjust(wspace = 0.3)
fig.tight_layout()
ax3.grid(which='minor', alpha=0.85)
ax3.grid(which='major', alpha=0.95, color='black')
#ax1.scatter(xses, js_divs_all_basic_DY, label='light jets nominal'   , s=100 )
#ax1.scatter(xses, js_divs_all_adv_DY, label='light jets adversarial' , s=100 )
#ax2.scatter(xses, js_divs_all_basic_Wc, label='c jets nominal'       , s=100 )
#ax2.scatter(xses, js_divs_all_adv_Wc, label='c jets adversarial'     , s=100 )
ax3.scatter(xses, js_divs_all_basic_TT, label='Nominal'       , s=200 )
ax3.scatter(xses, js_divs_all_adv_TT, label='Adversarial'     , s=200 )
#ax1.legend(frameon=True)
#ax2.legend(frameon=True)
ax3.legend(frameon=True, framealpha=1, title='b jets, DeepJet')
leg = ax3.get_legend()
leg._legend_box.align = "left"
leg.get_frame().set_linewidth(0.0)
#ax1.set_ylabel('JS divergence (a.u.)')
#ax2.set_ylabel('JS divergence (a.u.)')
ax3.set_ylabel('JS divergence (a.u.)')
hep.cms.label('Preliminary', data=True, lumi=41.5, year=2017)###fig.suptitle('JS divergence. Best epoch.', y=0.95)
fig.savefig('js_best_epoch_custom_DeepJet_BOTTOM_v2.pdf', bbox_inches='tight')
#fig.savefig('js_best_epoch_custom_DeepJet_BOTTOM_v2_30bins.pdf', bbox_inches='tight')

In [ ]:
#fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,8))
fig, ax3 = plt.subplots(1, figsize=(9,8))
#fig.subplots_adjust(wspace = 0.3)
fig.tight_layout()
ax3.grid(which='minor', alpha=0.85)
ax3.grid(which='major', alpha=0.95, color='black')
#ax1.scatter(xses, js_divs_all_basic_DY, label='light jets nominal'   , s=100 )
#ax1.scatter(xses, js_divs_all_adv_DY, label='light jets adversarial' , s=100 )
#ax2.scatter(xses, js_divs_all_basic_Wc, label='c jets nominal'       , s=100 )
#ax2.scatter(xses, js_divs_all_adv_Wc, label='c jets adversarial'     , s=100 )
ax3.scatter(xses, chi2_all_basic_TT, label='Nominal'       , s=200 )
ax3.scatter(xses, chi2_all_adv_TT, label='Adversarial'     , s=200 )
#ax1.legend(frameon=True)
#ax2.legend(frameon=True)
ax3.legend(frameon=True, framealpha=1, title='b jets, DeepJet')
leg = ax3.get_legend()
leg._legend_box.align = "left"
leg.get_frame().set_linewidth(0.0)
#ax1.set_ylabel('JS divergence (a.u.)')
#ax2.set_ylabel('JS divergence (a.u.)')
ax3.set_ylabel(r'$\chi^2$ (a.u.)')
hep.cms.label('Preliminary', data=True, lumi=41.5, year=2017)###fig.suptitle('JS divergence. Best epoch.', y=0.95)
fig.savefig('chi2_best_epoch_custom_DeepJet_BOTTOM_v2.pdf', bbox_inches='tight')
#fig.savefig('chi2_best_epoch_custom_DeepJet_BOTTOM_v2_30bins.pdf', bbox_inches='tight')

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,10))
fig.subplots_adjust(wspace = 0.3)
#fig.tight_layout()
ax1.scatter(xses, js_divs_all_basic_DY, label='DY nominal')
ax1.scatter(xses, js_divs_all_adv_DY, label='DY adversarial')
ax2.scatter(xses, js_divs_all_basic_Wc, label='Wc nominal')
ax2.scatter(xses, js_divs_all_adv_Wc, label='Wc adversarial')
ax3.scatter(xses, js_divs_all_basic_TT, label='TT nominal')
ax3.scatter(xses, js_divs_all_adv_TT, label='TT adversarial')
ax1.legend()
ax2.legend()
ax3.legend()
ax1.set_ylabel('JS divergence (a.u.)')
ax2.set_ylabel('JS divergence (a.u.)')
ax3.set_ylabel('JS divergence (a.u.)')
fig.suptitle('JS divergence. Best epoch.', y=0.95)
fig.savefig('js_best_epoch_custom_DeepJet.pdf')
#fig.savefig('js_best_epoch_custom_DeepJet_30bins.pdf')

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,10))
fig.subplots_adjust(wspace = 0.3)
#fig.tight_layout()
ax1.scatter(xses, kl_divs_all_basic_DY, label='DY nominal')
ax1.scatter(xses, kl_divs_all_adv_DY, label='DY adversarial')
ax2.scatter(xses, kl_divs_all_basic_Wc, label='Wc nominal')
ax2.scatter(xses, kl_divs_all_adv_Wc, label='Wc adversarial')
ax3.scatter(xses, kl_divs_all_basic_TT, label='TT nominal')
ax3.scatter(xses, kl_divs_all_adv_TT, label='TT adversarial')
ax1.legend()
ax2.legend()
ax3.legend()
ax1.set_ylabel('KL divergence (a.u.)')
ax2.set_ylabel('KL divergence (a.u.)')
ax3.set_ylabel('KL divergence (a.u.)')
fig.suptitle('KL divergence. Best epoch.', y=0.95)
fig.savefig('kl_best_epoch_custom_DeepJet.pdf')
#fig.savefig('kl_best_epoch_custom_DeepJet_30bins.pdf')

$\chi$

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(24,10))
fig.subplots_adjust(wspace = 0.3)
#fig.tight_layout()
ax1.scatter(xses, chi2_all_basic_DY, label='DY nominal')
ax1.scatter(xses, chi2_all_adv_DY, label='DY adversarial')
ax2.scatter(xses, chi2_all_basic_Wc, label='Wc nominal')
ax2.scatter(xses, chi2_all_adv_Wc, label='Wc adversarial')
ax3.scatter(xses, chi2_all_basic_TT, label='TT nominal')
ax3.scatter(xses, chi2_all_adv_TT, label='TT adversarial')
ax1.legend()
ax2.legend()
ax3.legend()
ax1.set_ylabel(r'$\chi^2$ (a.u.)')
ax2.set_ylabel(r'$\chi^2$ (a.u.)')
ax3.set_ylabel(r'$\chi^2$ (a.u.)')
fig.suptitle(r'$\mathcal{\chi^2}$. Best epoch.', y=0.95)
fig.savefig('js_best_epoch_custom_DeepJet.pdf')
#fig.savefig('chi2_best_epoch_custom_DeepJet_30bins.pdf')

In [ ]:
#!tar czf JS_divs_2022_09_15.tar.gz JS_divergences/*.svg